In [1]:
# module import
import os
import sys
from datetime import datetime
import pickle

import numpy as np
import PIL.Image
import torch
import torchvision
                
sys.path.append('../cnn_preferred')
from utils import normalise_img, clip_extreme_pixel,  get_cnn_features, img_deprocess, get_target_feature_shape
from activation_maximization import generate_preferred

In [2]:
## load network
net = torchvision.models.vgg16(pretrained=True)
net.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [3]:
# image mean and std for pre/de-process image for input network
img_mean=np.array([0.485, 0.456, 0.406],dtype=np.float),
img_std = np.array([0.229,0.224,0.225])

# if the model input is for 0-1 range, norm = 255, elif 0-255, norm = 1
norm = 255

In [4]:
# create save_dir
save_dir = '../result'
save_folder = 'jupyter_demo_torch_simpleCNN_conv'
save_folder = save_folder + '_' + datetime.now().strftime('%Y%m%dT%H%M%S')
save_path = os.path.join(save_dir,save_folder)
os.makedirs(save_path, exist_ok=True)

In [5]:
# initial image for the optimization
h, w = 224,224
initial_input = np.random.randint(0, 256, (h,w,3))

In [6]:
net

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [17]:
target_layer = "features[8]"
target_layer = "features[10]"

In [18]:
#target layer setting
target_layer_list = [target_layer]
## obtain target feature shape
# transform input shape for torch avairable shape
initial_torch_input = torch.Tensor(initial_input.transpose(2,0, 1)[np.newaxis])
# obtain target layer activation 
feat_shape = get_target_feature_shape(net, initial_torch_input, exec_str_list)

In [19]:
feat_shape

(1, 256, 56, 56)

In [20]:
# options
opts = {
    'img_mean': img_mean, # img_mean to preprocessing input image (the default is [0.485, 0.456, 0.406]) 
    'img_std': img_std,   # img_std to preprocessing input image  (the default is [0.229,0.224,0.225]) 
    'norm': norm,         # if the model input is for 0-1 range, norm = 255, elif 0-255, norm = 1 (defalt is 255)
    
    'iter_n': 200, # the total number of iterations for gradient descend (defalt is 200)

    'disp_every': 1, # display the information on the terminal for every n iterations (default is 1)

    'save_intermediate': True, # save the intermediate or not (default is None)
    'save_intermediate_every': 10, # save the intermediate for every n iterations (default is 10)
    'save_intermediate_path': save_path, # the path to save the intermediate (default is None)

    'lr_start': 1., # learning rate (default is 1.)
    'lr_end': 1.,   # we can change learning rate linearly setteing these two parameters 

    'momentum_start': 0.001, # gradient with momentum (default is 0.001)
    'momentum_end': 0.001,   # we can change momentum linearly setteing these two parameters too 

    'decay_start': 0.001, # pixel decay for each iteration (default is 0.001)
    'decay_end': 0.001,   # we can also change pixel decay linealy  

    'image_blur': True, # Use image smoothing or not (default is True)
    'sigma_start': 2.5, # the size of the gaussian filter for image smoothing (default is 2.5)
    'sigma_end': 0.5,   

    'image_jitter': True, # use image jittering during optimization (default is True)
    'jitter_size': 4,     # the size of jitter (default is 32)
    
    'use_p_norm_reg': False, # use p_norm regularization (default is False)
    'p': 2,

    'use_TV_norm_reg': False, # use total variance norm (default is False)
    'TVbeta1': 1,             # the order of  spatial domain
    'TVbeta2':1.2,            # the order temporal domain (for video input)

    'clip_small_norm': True,   # clip or not the pixels with extreme high or low value (default True) 
    'clip_small_norm_every': 1,
    'n_pct_start': 5,
    'n_pct_end': 5,

    'clip_small_contribution': True, # clip or not the poxels with smal contribution norm of RGB channels
    'clip_small_contribution_every': 1,
    'c_pct_start': 5,
    'c_pct_end':5,
    
    'initial_input': initial_input, # the initial image for the optimization (setting to None will use random noise as initial image)
    }



In [21]:
# set channel index and center postion (for setting feature mask)
channel_list = [14,56]
y_index = int(feat_shape[2]/2)
x_index = int(feat_shape[3]/2)

In [22]:
for channel in channel_list:
    #
    print('')
    print('channel='+str(channel))
    print('')
    
    # Instead to setting target channel, generate_preferred function also accepts feature mask, which
    # The values of the mask array are binary, (1: target uint; 0: irrelevant unit) and whose shape is 
    # the same as that of target layer
    
    #create feature_mask
    feature_mask = np.zeros(feat_shape)
    # Only try to maximize the center of unit
    feature_mask[0,channel, y_index, x_index] = 1.
        
    # activation maximization
    preferred_stim = generate_preferred(net, target_layer_list, feature_mask=feature_mask, **opts)
    # save the results
    save_name = 'preferred_img' + '_layer_' + str(target_layer) + '_channel_' + str(channel) + '.npy'
    np.save(os.path.join(save_path,save_name), preferred_stim)

    save_name = 'preferred_img' + '_layer_' + str(target_layer) + '_channel_' + str(channel) + '.jpg'
    # To better display the image, clip pixels with extreme values (0.02% of
    # pixels with extreme low values and 0.02% of the pixels with extreme high
    # values). And then normalise the image by mapping the pixel value to be
    # within [0,255].
    PIL.Image.fromarray(normalise_img(clip_extreme_pixel(preferred_stim, pct=0.04))).save(
                    os.path.join(save_path, save_name))


channel=14

iter=1; mean(abs(feat))=5.89461;
iter=2; mean(abs(feat))=1498.56;
iter=3; mean(abs(feat))=2739.8;
iter=4; mean(abs(feat))=5285.72;
iter=5; mean(abs(feat))=2645.55;
iter=6; mean(abs(feat))=2517.29;
iter=7; mean(abs(feat))=2974.32;
iter=8; mean(abs(feat))=3380.27;
iter=9; mean(abs(feat))=379.454;
iter=10; mean(abs(feat))=4587.21;
iter=11; mean(abs(feat))=3953.31;
iter=12; mean(abs(feat))=7453.28;
iter=13; mean(abs(feat))=3751.87;
iter=14; mean(abs(feat))=5618.27;
iter=15; mean(abs(feat))=6989.64;
iter=16; mean(abs(feat))=5098.97;
iter=17; mean(abs(feat))=865.434;
iter=18; mean(abs(feat))=6793.17;
iter=19; mean(abs(feat))=3513.88;
iter=20; mean(abs(feat))=2557.05;
iter=21; mean(abs(feat))=2481.68;
iter=22; mean(abs(feat))=3904.78;
iter=23; mean(abs(feat))=6286.18;
iter=24; mean(abs(feat))=3596.25;
iter=25; mean(abs(feat))=4149.51;
iter=26; mean(abs(feat))=4837.01;
iter=27; mean(abs(feat))=7323.94;
iter=28; mean(abs(feat))=3170.42;
iter=29; mean(abs(feat))=5891.7;
iter=30; mea

iter=41; mean(abs(feat))=775.452;
iter=42; mean(abs(feat))=477.644;
iter=43; mean(abs(feat))=283.371;
iter=44; mean(abs(feat))=1162.19;
iter=45; mean(abs(feat))=616.821;
iter=46; mean(abs(feat))=2147.86;
iter=47; mean(abs(feat))=4181.09;
iter=48; mean(abs(feat))=3136.54;
iter=49; mean(abs(feat))=1137.11;
iter=50; mean(abs(feat))=1215.34;
iter=51; mean(abs(feat))=2988.06;
iter=52; mean(abs(feat))=4153.95;
iter=53; mean(abs(feat))=4532.81;
iter=54; mean(abs(feat))=4590.33;
iter=55; mean(abs(feat))=2035.26;
iter=56; mean(abs(feat))=1537.59;
iter=57; mean(abs(feat))=81.499;
iter=58; mean(abs(feat))=441.165;
iter=59; mean(abs(feat))=723.348;
iter=60; mean(abs(feat))=1920.54;
iter=61; mean(abs(feat))=1121.41;
iter=62; mean(abs(feat))=3966.17;
iter=63; mean(abs(feat))=2594.01;
iter=64; mean(abs(feat))=751.861;
iter=65; mean(abs(feat))=241.192;
iter=66; mean(abs(feat))=2083.33;
iter=67; mean(abs(feat))=1768.77;
iter=68; mean(abs(feat))=4140.32;
iter=69; mean(abs(feat))=603.097;
iter=70; mean(a